# Quick Start in 30s

## Build the index

In [ ]:
from leann.api import LeannBuilder

builder = LeannBuilder(backend_name="hnsw")
builder.add_text("C# is a powerful programming language")
builder.add_text("Python is a powerful programming language and it is very popular")
builder.add_text("Machine learning transforms industries")
builder.add_text("Neural networks process complex data")
builder.add_text("Leann is a great storage saving engine for RAG on your MacBook")
builder.build_index("knowledge.leann")

## Search with real-time embeddings

In [ ]:
from leann.api import LeannSearcher

searcher = LeannSearcher("knowledge.leann")
results = searcher.search("programming languages", top_k=2)
results

## Chat with LEANN using retrieved results

In [ ]:
from leann.api import LeannChat

llm_config = {
    "type": "ollama",
    "model": "llama3.2:1b"
}

chat = LeannChat(index_path="knowledge.leann", llm_config=llm_config)
response = chat.ask(
    "Compare the two retrieved programming languages and say which one is more popular today.",
    top_k=2,
)
response